In [1]:
import time
import torch
import transformers

import numpy as np
import pandas as pd

from tqdm import tqdm
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import BlenderbotSmallTokenizer, BlenderbotSmallForConditionalGeneration, \
TrainingArguments, Trainer

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'

### Check out the Dataset

In [3]:
df = pd.read_csv('news-expand.csv')

In [4]:
df.head()

,Unnamed: 0,summary,text
0,0,Australian batsman Steve Smith reveals he near...,australia batsman steve smith has revealed tha...
1,1,milind soman recently trekked to the highest p...,milind soman needs no introduction and nor doe...
2,2,singer Aditya Narayan shared a picture of hims...,ahead of his wedding (reportedly on december 1...
3,3,Rubina Dilaik will reveal one of the deepest s...,tv star rubina dilaik will reveal one of the d...
4,4,"the couple, who got married exactly a month ag...",it is a happy day for kajal aggarwal and her h...


### Create CustomDataset class for training

In [5]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.summary
        self.ctext = self.data.text

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length = self.source_len, padding = 'max_length',\
                                                  return_tensors = 'pt', truncation = True)
        target = self.tokenizer.batch_encode_plus([text], max_length = self.summ_len, padding = 'max_length',\
                                                  return_tensors = 'pt', truncation = True)

        source_ids = source['input_ids'].squeeze().to(dtype = torch.long)
        source_mask = source['attention_mask'].squeeze().to(dtype = torch.long)
        target_ids = target['input_ids'].squeeze().to(dtype = torch.long)
        
        y_ids = target_ids[:-1].contiguous() # make y_ids contiguous 
        lm_labels = target_ids[1:].clone().detach() # make fast copy
        lm_labels[target_ids[1:] == tokenizer.pad_token_id] = -100 # replace pad tokens

        return {
            'input_ids': source_ids, 
            'attention_mask': source_mask, 
            'decoder_input_ids': y_ids,
            'labels': lm_labels
        }

In [6]:
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2 
TRAIN_EPOCHS = 25      
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4    
SEED = 42               
MAX_LEN = 512
SUMMARY_LEN = 128

torch.manual_seed(SEED) 
np.random.seed(SEED) 
torch.backends.cudnn.deterministic = True

tokenizer = BlenderbotSmallTokenizer.from_pretrained("facebook/blenderbot_small-90M")

df = df[['text','summary']]
df.summary = 'summarize: ' + df.summary 

train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state = SEED)
val_dataset = df.drop(train_dataset.index).reset_index(drop = True)
train_dataset = train_dataset.reset_index(drop = True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

train_params = {
    'batch_size': TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

model = BlenderbotSmallForConditionalGeneration.from_pretrained("facebook/blenderbot_small-90M")
model = model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr = LEARNING_RATE)

FULL Dataset: (8104, 2)
TRAIN Dataset: (6483, 2)
TEST Dataset: (1621, 2)


In [7]:
args = TrainingArguments(output_dir="blenderbot_small-news",
                         seed=42,
                         num_train_epochs=10,
                         per_device_train_batch_size=4,  
                         # max batch size without OOM exception, because of the large max token length
                         per_device_eval_batch_size=4,
                         logging_steps=2500,
                         save_steps=0,
                        )

In [8]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=training_set,
    eval_dataset=val_set,
)

In [9]:
trainer.train()

Step,Training Loss


RuntimeError: CUDA out of memory. Tried to allocate 108.00 MiB (GPU 0; 9.78 GiB total capacity; 3.48 GiB already allocated; 59.81 MiB free; 3.53 GiB reserved in total by PyTorch)

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    texts = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(loader, 0)):
            y = data['decoder_input_ids'].to(device, dtype = torch.long)
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length = 100, 
                num_beams = 2,
                repetition_penalty = 2.5, 
                length_penalty = 1.0, 
                early_stopping = True
                )
            preds = [tokenizer.decode(g, skip_special_tokens = True, clean_up_tokenization_spaces = True)\
                     for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens = True, clean_up_tokenization_spaces = True)\
                      for t in y]
            text = [tokenizer.decode(i, skip_special_tokens = True, clean_up_tokenization_spaces = True)\
                      for i in ids]
            if _%2500==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
            texts.extend(text)
    return predictions, actuals, texts

In [ ]:
trainer.save_model("t5-tt-trainer/")

In [ ]:
start_time = time.time()
for epoch in range(VAL_EPOCHS):
    predictions, actuals, text = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text': predictions,'Actual Text': actuals, 'Text': text})
print("Validation took " + str(time.time() - start_time) + " seconds")

### Check out Generated vs Actual Text

In [ ]:
final_df

In [ ]:
final_df.to_csv('predsvsactual.csv')

### Save Model in tensorflow to upload to huggingface model repository

In [ ]:
import transformers

In [ ]:
tf_model = transformers.TFT5ForConditionalGeneration.from_pretrained("t5-tt-trainer/", from_pt=True)

In [ ]:
tf_model.save_pretrained("t5-tt-trainer/")

In [ ]:
tokenizer = transformers.T5Tokenizer.from_pretrained("t5-base")

In [ ]:
tokenizer.save_pretrained('./')